# Basketball Shot Inference and Trajectory Suggestion

This notebook performs inference on up to 10 missed shots from the test dataset (`test.csv`) and suggests optimal shooting trajectories to improve the chances of making the shot. It uses a trained PyTorch LSTM model (`shot_model.pth`) to predict shot outcomes and generates visualizations for each missed shot.

## Objectives
- Load and preprocess the test dataset.
- Identify up to 10 missed shots.
- Suggest optimal shooting trajectories for each missed shot.
- Visualize the original and suggested trajectories.

## Outputs
- Logs: `infer.log` with inference results and suggestions.
- Plots: `shot_{shot_id}_feedback.png` for each processed missed shot, showing position changes and trajectory comparisons.


In [20]:
# Imports and Setup
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import ast
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from google.colab import drive
import os
import logging
import pickle
import sys
import torch
import torch.nn as nn

# Mount Google Drive
try:
    drive.mount('/content/drive')
    drive_mounted = True
except Exception as e:
    print(f"Failed to mount Google Drive: {e}")
    drive_mounted = False

# Define paths
if drive_mounted and os.path.exists('/content/drive/MyDrive/smai_project/'):
    data_path = '/content/drive/MyDrive/smai_project/dataset/dataset15/'
    output_path = '/content/drive/MyDrive/smai_project/output_graphs/'
    model_path = '/content/drive/MyDrive/smai_project/model/model15/'
else:
    print("Google Drive not accessible. Using local directory.")
    data_path = '/content/'
    output_path = '/content/output_graphs/'
    model_path = '/content/model/'

# Create output directory if it doesn't exist
try:
    if not os.path.exists(output_path):
        os.makedirs(output_path)
except Exception as e:
    print(f"Failed to create output directory {output_path}: {e}")
    output_path = '/content/'  # Fallback to a writable directory

log_file_path = os.path.join(output_path, 'infer.log')

# Set up custom logger
logger = logging.getLogger('ShotInference')
logger.setLevel(logging.INFO)
logger.handlers = []  # Remove any existing handlers to prevent duplicates

# Create and configure FileHandler with line buffering
try:
    file_handler = logging.FileHandler(log_file_path, buffering=1)  # Enable line buffering
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
    logger.addHandler(file_handler)
except Exception as e:
    print(f"Failed to set up FileHandler for {log_file_path}: {e}")

# Create and configure StreamHandler for console output
stream_handler = logging.StreamHandler(sys.stdout)
stream_handler.setLevel(logging.INFO)
stream_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
logger.addHandler(stream_handler)

print("Logging setup complete. Starting inference.")
for handler in logger.handlers:
    handler.flush()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Failed to set up FileHandler for /content/drive/MyDrive/smai_project/output_graphs/infer.log: FileHandler.__init__() got an unexpected keyword argument 'buffering'
Logging setup complete. Starting inference.


In [21]:
# Define ShotPredictor Model Class
class ShotPredictor(nn.Module):
    """PyTorch model for basketball shot prediction."""
    def __init__(self, ball_input_size=3, static_input_size=20, hidden_size=64):
        super(ShotPredictor, self).__init__()
        self.static_input_size = static_input_size
        self.lstm1 = nn.LSTM(ball_input_size, hidden_size, batch_first=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size // 2, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)

        self.static_fc1 = nn.Linear(static_input_size, 32)
        self.static_relu = nn.ReLU()
        self.static_dropout = nn.Dropout(0.2)

        self.fc_combined = nn.Linear(hidden_size // 2 + 32, 32)
        self.relu_combined = nn.ReLU()
        self.dropout_combined = nn.Dropout(0.2)
        self.fc_out = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, ball_seq, static_features):
        # Verify static features shape
        if static_features.size(-1) != self.static_input_size:
            raise ValueError(f"Expected static_features size {self.static_input_size}, got {static_features.size(-1)}")

        # Ball sequence processing
        lstm_out, _ = self.lstm1(ball_seq)
        lstm_out = self.dropout1(lstm_out)
        lstm_out, _ = self.lstm2(lstm_out)
        lstm_out = self.dropout2(lstm_out)
        lstm_out = lstm_out[:, -1, :]  # Take the last time step

        # Static features processing
        static_out = self.static_fc1(static_features)
        static_out = self.static_relu(static_out)
        static_out = self.static_dropout(static_out)

        # Combine
        combined = torch.cat((lstm_out, static_out), dim=1)
        out = self.fc_combined(combined)
        out = self.relu_combined(out)
        out = self.dropout_combined(out)
        out = self.fc_out(out)
        out = self.sigmoid(out)
        return out

In [22]:
# Feature Extraction Function
def extract_sequence_features(df):
    """Extract features from ball and player sequences."""
    def get_shooter_position(ball_seq, players_seq, personId):
        shooter_x, shooter_y = None, None
        if players_seq and isinstance(players_seq, list) and len(players_seq) > 0:
            first_frame = players_seq[0] if isinstance(players_seq[0], list) else players_seq
            if first_frame and isinstance(first_frame, list):
                for player in first_frame:
                    if len(player) != 5:
                        continue
                    _, player_id, x, y, _ = player
                    if player_id == personId:
                        shooter_x, shooter_y = x, y
                        break

        if shooter_x is None or shooter_y is None:
            if not ball_seq or not isinstance(ball_seq, list) or len(ball_seq) == 0:
                return 0, 0
            first_pos = ball_seq[0]
            if not isinstance(first_pos, list) or len(first_pos) != 3:
                return 0, 0
            shooter_x, shooter_y, _ = first_pos
        return shooter_x, shooter_y

    df['shooter_x'], df['shooter_y'] = zip(*df.apply(
        lambda row: get_shooter_position(row['ball_seq'], row['players_seq'], row['personId']), axis=1
    ))

    df['initial_height'] = df['ball_seq'].apply(lambda seq: seq[0][2] if len(seq) > 0 and len(seq[0]) == 3 else 0)
    df['max_height'] = df['ball_seq'].apply(lambda seq: max([point[2] for point in seq]) if len(seq) > 0 and all(len(point) == 3 for point in seq) else 0)
    df['traj_length'] = df['ball_seq'].apply(len)
    df['shotDistance'] = df.apply(
        lambda row: np.sqrt((row['shooter_x'] - row['basket_x'])**2 + (row['shooter_y'] - row['basket_y'])**2), axis=1
    )
    df['shotDistance'] = df['shotDistance'].replace(0, 1)
    df['traj_curvature'] = df['max_height'] / df['shotDistance']

    df['release_angle'] = df.apply(
        lambda row: np.arctan2(row['shooter_y'] - row['basket_y'], row['shooter_x'] - row['basket_x']), axis=1
    )

    def calculate_defender_proximity(shooter_x, shooter_y, players_seq):
        min_distance = float('inf')
        shooter_team_id = None

        if not players_seq or not isinstance(players_seq, list) or len(players_seq) == 0:
            return 0

        first_frame = players_seq[0] if isinstance(players_seq[0], list) else players_seq
        if not first_frame or not isinstance(first_frame, list):
            return 0

        closest_player_distance = float('inf')
        for player in first_frame:
            if len(player) != 5:
                continue
            team_id, _, x, y, _ = player
            distance = np.sqrt((shooter_x - x)**2 + (shooter_y - y)**2)
            if distance < closest_player_distance:
                closest_player_distance = distance
                shooter_team_id = team_id

        if shooter_team_id is None:
            logger.warning(f"Could not determine shooter team for position ({shooter_x}, {shooter_y})")
            for handler in logger.handlers:
                handler.flush()
            return 0

        for player in first_frame:
            if len(player) != 5:
                continue
            team_id, _, x, y, _ = player
            if team_id == shooter_team_id:
                continue
            distance = np.sqrt((shooter_x - x)**2 + (shooter_y - y)**2)
            min_distance = min(min_distance, distance)
        return min_distance if min_distance != float('inf') else 0

    df['defender_proximity'] = df.apply(
        lambda row: calculate_defender_proximity(row['shooter_x'], row['shooter_y'], row['players_seq']), axis=1
    )

    def extract_player_positions(players_seq, personId, shooter_x, shooter_y):
        shooter_team_id = None
        teammates = []
        defenders = []

        if not players_seq or not isinstance(players_seq, list) or len(players_seq) == 0:
            return [0, 0, 0, 0]

        first_frame = players_seq[0] if isinstance(players_seq[0], list) else players_seq
        if not first_frame or not isinstance(first_frame, list):
            return [0, 0, 0, 0]

        closest_player_distance = float('inf')
        for player in first_frame:
            if len(player) != 5:
                continue
            team_id, _, x, y, _ = player
            distance = np.sqrt((shooter_x - x)**2 + (shooter_y - y)**2)
            if distance < closest_player_distance:
                closest_player_distance = distance
                shooter_team_id = team_id

        if shooter_team_id is None:
            logger.warning(f"Could not determine shooter team for position ({shooter_x}, {shooter_y})")
            for handler in logger.handlers:
                handler.flush()
            return [0, 0, 0, 0]

        for player in first_frame:
            if len(player) != 5:
                continue
            team_id, _, x, y, _ = player
            if team_id == shooter_team_id:
                teammates.append([x, y])
            else:
                defenders.append([x, y])

        closest_teammate = [0, 0]
        if teammates:
            teammate_distances = [(np.sqrt((shooter_x - tx)**2 + (shooter_y - ty)**2), tx, ty) for tx, ty in teammates]
            teammate_distances.sort()
            closest_teammate = [teammate_distances[1][1], teammate_distances[1][2]] if len(teammate_distances) > 1 else teammate_distances[0][1:3]

        closest_defender = [0, 0]
        if defenders:
            defender_distances = [(np.sqrt((shooter_x - dx)**2 + (shooter_y - dy)**2), dx, dy) for dx, dy in defenders]
            defender_distances.sort()
            closest_defender = [defender_distances[0][1], defender_distances[0][2]]

        return closest_teammate + closest_defender

    df[['teammate_x', 'teammate_y', 'defender_x', 'defender_y']] = df.apply(
        lambda row: extract_player_positions(row['players_seq'], row['personId'], row['shooter_x'], row['shooter_y']), axis=1, result_type='expand'
    )

    def extract_ball_dynamics(ball_seq):
        if not ball_seq or len(ball_seq) < 3:
            return [0, 0, 0, 0, 0, 0]

        velocities = []
        for i in range(1, len(ball_seq)):
            if len(ball_seq[i]) != 3 or len(ball_seq[i-1]) != 3:
                continue
            dx = ball_seq[i][0] - ball_seq[i-1][0]
            dy = ball_seq[i][1] - ball_seq[i-1][1]
            dz = ball_seq[i][2] - ball_seq[i-1][2]
            velocities.append([dx, dy, dz])

        if not velocities:
            return [0, 0, 0, 0, 0, 0]

        accelerations = []
        for i in range(1, len(velocities)):
            ax = velocities[i][0] - velocities[i-1][0]
            ay = velocities[i][1] - velocities[i-1][1]
            az = velocities[i][2] - velocities[i-1][2]
            accelerations.append([ax, ay, az])

        if not accelerations:
            avg_vel_x = np.mean([v[0] for v in velocities])
            avg_vel_y = np.mean([v[1] for v in velocities])
            avg_vel_z = np.mean([v[2] for v in velocities])
            return [avg_vel_x, avg_vel_y, avg_vel_z, 0, 0, 0]

        avg_vel_x = np.mean([v[0] for v in velocities])
        avg_vel_y = np.mean([v[1] for v in velocities])
        avg_vel_z = np.mean([v[2] for v in velocities])
        avg_acc_x = np.mean([a[0] for a in accelerations])
        avg_acc_y = np.mean([a[1] for a in accelerations])
        avg_acc_z = np.mean([a[2] for a in accelerations])

        return [avg_vel_x, avg_vel_y, avg_vel_z, avg_acc_x, avg_acc_y, avg_acc_z]

    df[['avg_vel_x', 'avg_vel_y', 'avg_vel_z', 'avg_acc_x', 'avg_acc_y', 'avg_acc_z']] = df['ball_seq'].apply(
        lambda seq: extract_ball_dynamics(seq)
    ).apply(pd.Series)

    def extract_shot_arc(ball_seq, basket_x, basket_y):
        if not ball_seq or len(ball_seq) < 3:
            return [0, 0]

        try:
            release_x, release_y, release_z = ball_seq[0]
            peak_idx = max(range(len(ball_seq)), key=lambda i: ball_seq[i][2] if len(ball_seq[i]) == 3 else 0)
            peak_x, peak_y, peak_z = ball_seq[peak_idx]

            last_valid_idx = -1
            for i in range(len(ball_seq)-1, 0, -1):
                if len(ball_seq[i]) == 3:
                    last_valid_idx = i
                    break

            if last_valid_idx <= 0:
                return [0, 0]

            p2 = ball_seq[last_valid_idx]
            p1 = ball_seq[max(0, last_valid_idx-1)]

            if len(p1) != 3 or len(p2) != 3:
                return [0, 0]

            horizontal_dist = np.sqrt((p2[0] - p1[0])**2 + (p2[1] - p1[1])**2)
            entry_angle = 90 if horizontal_dist == 0 else np.degrees(np.arctan2(p2[2] - p1[2], horizontal_dist))

            total_dist = np.sqrt((basket_x - release_x)**2 + (basket_y - release_y)**2)
            arc_height_ratio = 0 if total_dist == 0 else peak_z / total_dist

            return [entry_angle, arc_height_ratio]

        except Exception as e:
            logger.error(f"Error extracting shot arc: {e}")
            for handler in logger.handlers:
                handler.flush()
            return [0, 0]

    df[['entry_angle', 'arc_height_ratio']] = df.apply(
        lambda row: extract_shot_arc(row['ball_seq'], row['basket_x'], row['basket_y']),
        axis=1, result_type='expand'
    )

    return df


In [23]:
# Sequence Processing Function
def process_ball_sequences(df):
    """Process ball sequences for LSTM input."""
    sequences = []
    labels = []

    for _, row in df.iterrows():
        ball_seq = row['ball_seq']
        result = row['shotResult']

        if len(ball_seq) > 37:
            ball_seq = ball_seq[:37]
        elif len(ball_seq) < 37:
            last_pos = ball_seq[-1] if ball_seq else [0, 0, 0]
            while len(ball_seq) < 37:
                ball_seq.append(last_pos)

        sequence = []
        for moment in ball_seq:
            if isinstance(moment, list) and len(moment) == 3:
                sequence.append(moment)
            else:
                sequence.append([0, 0, 0])

        sequences.append(sequence)
        labels.append(result)

    return np.array(sequences), np.array(labels)


In [24]:
# Trajectory Generation and Suggestion Functions
def generate_alternative_trajectories(shooter_x, shooter_y, basket_x, basket_y, shot_distance, current_angle, current_max_height, current_features, max_distance=5):
    """Generate alternative shooting trajectories."""
    alternatives = []
    angle_range = np.deg2rad(np.linspace(-30, 30, 5))
    angles = current_angle + angle_range
    height_range = np.linspace(-2, 2, 5)
    max_heights = current_max_height + height_range
    entry_angle_range = np.linspace(-15, 15, 3)
    arc_ratio_range = np.linspace(-0.1, 0.2, 3)

    for angle in angles:
        for max_height in max_heights:
            for entry_angle_diff in entry_angle_range:
                for arc_ratio_diff in arc_ratio_range:
                    if max_height <= 0:
                        continue
                    new_x = basket_x + shot_distance * np.cos(angle)
                    new_y = basket_y + shot_distance * np.sin(angle)
                    distance_moved = np.sqrt((new_x - shooter_x)**2 + (new_y - shooter_y)**2)
                    if distance_moved > max_distance:
                        continue

                    new_features = current_features.copy()
                    new_features[0] = new_x
                    new_features[1] = new_y
                    new_features[2] = angle
                    new_features[4] = max_height
                    new_features[6] = max_height / shot_distance

                    if len(current_features) >= 20:
                        current_entry_angle = current_features[18]
                        current_arc_ratio = current_features[19]
                        new_features[18] = current_entry_angle + entry_angle_diff
                        new_features[19] = current_arc_ratio + arc_ratio_diff

                    alternatives.append({
                        'x': new_x,
                        'y': new_y,
                        'angle': angle,
                        'max_height': max_height,
                        'entry_angle': current_features[18] + entry_angle_diff if len(current_features) >= 19 else 0,
                        'arc_ratio': current_features[19] + arc_ratio_diff if len(current_features) >= 20 else 0,
                        'features': new_features
                    })
    return alternatives

def suggest_optimal_trajectory(missed_shot_features, missed_shot_ball_seq, basket_x, basket_y, model, scaler_static, scaler_ball, device, max_distance=5):
    """Suggest an optimal shooting trajectory for a missed shot."""
    shooter_x = missed_shot_features[0]
    shooter_y = missed_shot_features[1]
    current_angle = missed_shot_features[2]
    current_max_height = missed_shot_features[4]
    shot_distance = np.sqrt((shooter_x - basket_x)**2 + (shooter_y - basket_y)**2)
    if shot_distance == 0:
        shot_distance = 1

    alternatives = generate_alternative_trajectories(
        shooter_x, shooter_y, basket_x, basket_y, shot_distance, current_angle, current_max_height, missed_shot_features, max_distance
    )

    if not alternatives:
        return None, None, f"No viable alternative trajectories found within {max_distance} units."

    best_score = -1
    best_alternative = None
    for alt in alternatives:
        alt_features = alt['features']
        alt_features_scaled = scaler_static.transform([alt_features])
        ball_seq_scaled = missed_shot_ball_seq.reshape(1, 37, 3)
        with torch.no_grad():
            ball_seq_tensor = torch.FloatTensor(ball_seq_scaled).to(device)
            alt_features_tensor = torch.FloatTensor(alt_features_scaled).to(device)
            output = model(ball_seq_tensor, alt_features_tensor)
            score = torch.sigmoid(output).item()
        if score > best_score:
            best_score = score
            best_alternative = alt

    if best_alternative is None:
        return None, None, "No better trajectory found."

    suggestion = (f"Original position (x: {shooter_x:.2f}, y: {shooter_y:.2f}), "
                  f"Move to position (x: {best_alternative['x']:.2f}, y: {best_alternative['y']:.2f}), "
                  f"use a release angle of {np.degrees(best_alternative['angle']):.2f} degrees, "
                  f"aim for a maximum trajectory height of {best_alternative['max_height']:.2f} feet, "
                  f"and try for an entry angle of {best_alternative['entry_angle']:.2f} degrees "
                  f"for a {best_score*100:.2f}% chance of making the shot.")
    return best_alternative, best_score, suggestion


In [25]:
# Load and Preprocess Test Data
print("Loading test dataset...")
for handler in logger.handlers:
    handler.flush()
test_df = pd.read_csv(data_path + 'test.csv')

test_df['players_seq'] = test_df['players_seq'].apply(ast.literal_eval)
test_df['ball_seq'] = test_df['ball_seq'].apply(ast.literal_eval)

test_df.drop(test_df[~test_df['shotResult'].isin(['Made Shot', 'Missed Shot'])].index, inplace=True)
test_df['shotResult'] = test_df['shotResult'].map({'Made Shot': 1, 'Missed Shot': 0})

print("Extracting features...")
for handler in logger.handlers:
    handler.flush()
test_df = extract_sequence_features(test_df)

static_features = [
    'shooter_x', 'shooter_y', 'release_angle', 'initial_height', 'max_height',
    'traj_length', 'traj_curvature', 'defender_proximity',
    'teammate_x', 'teammate_y', 'defender_x', 'defender_y',
    'avg_vel_x', 'avg_vel_y', 'avg_vel_z', 'avg_acc_x', 'avg_acc_y', 'avg_acc_z',
    'entry_angle', 'arc_height_ratio'
]

test_df[static_features] = test_df[static_features].replace([np.inf, -np.inf], 0).fillna(0)

print("Processing ball sequences...")
for handler in logger.handlers:
    handler.flush()
X_test_ball, y_test = process_ball_sequences(test_df)
X_test_ball = X_test_ball.reshape(X_test_ball.shape[0], 37, 3)

# Load model and scalers
model_path_file = os.path.join(model_path, 'shot_model.pth')
scaler_ball_path = os.path.join(model_path, 'scaler_ball.pkl')
scaler_static_path = os.path.join(model_path, 'scaler_static.pkl')

if not os.path.exists(model_path_file):
    raise FileNotFoundError(f"Model file 'shot_model.pth' not found at {model_path_file}")
if not os.path.exists(scaler_ball_path):
    raise FileNotFoundError(f"Scaler file 'scaler_ball.pkl' not found at {scaler_ball_path}")
if not os.path.exists(scaler_static_path):
    raise FileNotFoundError(f"Scaler file 'scaler_static.pkl' not found at {scaler_static_path}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
for handler in logger.handlers:
    handler.flush()

print("Loading model and scalers...")
for handler in logger.handlers:
    handler.flush()
model = ShotPredictor(ball_input_size=3, static_input_size=len(static_features)).to(device)
model.load_state_dict(torch.load(model_path_file, map_location=device))
model.eval()

with open(scaler_ball_path, 'rb') as f:
    scaler_ball = pickle.load(f)
with open(scaler_static_path, 'rb') as f:
    scaler_static = pickle.load(f)

# Normalize test data
X_test_ball_2d = X_test_ball.reshape(-1, 3)
X_test_ball_2d = scaler_ball.transform(X_test_ball_2d)
X_test_ball = X_test_ball_2d.reshape(X_test_ball.shape)

X_test_static = test_df[static_features].values
X_test_static = scaler_static.transform(X_test_static)


Loading test dataset...
Extracting features...
Processing ball sequences...
Using device: cuda
Loading model and scalers...


In [26]:
# Process Missed Shots and Generate Feedback/Visualizations
missed_shots = test_df[test_df['shotResult'] == 0]
num_shots_to_process = min(10, len(missed_shots))
print(f"Found {len(missed_shots)} missed shots. Processing {num_shots_to_process} shots.")
for handler in logger.handlers:
    handler.flush()

if num_shots_to_process > 0:
    for i in range(num_shots_to_process):
        row = missed_shots.iloc[i]
        shot_id = missed_shots.index[i]
        missed_features = row[static_features].values
        ball_seq = X_test_ball[test_df.index.get_loc(shot_id)]
        basket_x, basket_y = row['basket_x'], row['basket_y']

        logger.info(f"Processing missed shot ID {shot_id}...")
        for handler in logger.handlers:
            handler.flush()

        # Correct basket position
        court_length = 94
        basket_positions = [(0, 25), (94, 25)]
        if row['shooter_x'] > court_length / 2:
            expected_basket_x, expected_basket_y = basket_positions[1]  # (94, 25)
        else:
            expected_basket_x, expected_basket_y = basket_positions[0]  # (0, 25)
        if (row['basket_x'], row['basket_y']) not in basket_positions:
            logger.warning(f"Basket position ({row['basket_x']}, {row['basket_y']}) adjusted to ({expected_basket_x}, {expected_basket_y})")
            basket_x, basket_y = expected_basket_x, expected_basket_y
        for handler in logger.handlers:
            handler.flush()

        best_alternative, best_score, suggestion = suggest_optimal_trajectory(
            missed_features, ball_seq, basket_x, basket_y, model, scaler_static, scaler_ball, device, max_distance=10
        )

        logger.info(f"Shot {shot_id}: {suggestion}")
        for handler in logger.handlers:
            handler.flush()

        if best_alternative:
            feedback_data = {
                'shot_id': shot_id,
                'orig_x': row['shooter_x'],
                'orig_y': row['shooter_y'],
                'sugg_x': best_alternative['x'],
                'sugg_y': best_alternative['y'],
                'angle': np.degrees(best_alternative['angle']),
                'orig_max_height': row['max_height'],
                'sugg_max_height': best_alternative['max_height'],
                'entry_angle': best_alternative['entry_angle'],
                'arc_ratio': best_alternative['arc_ratio'],
                'prob': best_score,
                'basket_x': basket_x,
                'basket_y': basket_y
            }

            # Visualize feedback
            court_length = 94
            court_width = 50

            fig = plt.figure(figsize=(14, 10))
            fig.suptitle(f'Shot {feedback_data["shot_id"]} Feedback: Position and Trajectory', fontsize=16)

            # 2D Plot: Top-down view of the court
            ax1 = fig.add_subplot(221)
            ax1.set_title('Position Change (2D)')
            ax1.set_xlabel('X (feet)')
            ax1.set_ylabel('Y (feet)')
            ax1.set_xlim(-10, court_length + 10)
            ax1.set_ylim(-5, court_width + 5)

            ax1.plot([0, court_length], [0, 0], 'k-')
            ax1.plot([0, court_length], [court_width, court_width], 'k-')
            ax1.plot([0, 0], [0, court_width], 'k-')
            ax1.plot([court_length, court_length], [0, court_width], 'k-')

            ax1.plot(basket_x, basket_y, 'ro', markersize=10, label='Basket')
            ax1.plot(feedback_data['orig_x'], feedback_data['orig_y'], 'bo', markersize=12, label='Original Position')
            ax1.scatter(feedback_data['sugg_x'], feedback_data['sugg_y'], c=[feedback_data['prob']], cmap='Greens', vmin=0, vmax=1, s=120,
                        label='Suggested Position', edgecolors='k')
            ax1.annotate('Orig', (feedback_data['orig_x'], feedback_data['orig_y']), textcoords="offset points", xytext=(0,10), ha='center')
            ax1.annotate('Sugg', (feedback_data['sugg_x'], feedback_data['sugg_y']), textcoords="offset points", xytext=(0,10), ha='center')
            ax1.arrow(feedback_data['orig_x'], feedback_data['orig_y'],
                      feedback_data['sugg_x'] - feedback_data['orig_x'], feedback_data['sugg_y'] - feedback_data['orig_y'],
                      head_width=1, head_length=1.5, fc='b', ec='b')

            ax1.legend()
            ax1.grid(True)

            # 3D Plot: Original trajectory
            ax2 = fig.add_subplot(222, projection='3d')
            ax2.set_title('Original Trajectory (3D)')
            ax2.set_xlabel('X (feet)')
            ax2.set_ylabel('Y (feet)')
            ax2.set_zlabel('Height (feet)')
            ax2.set_xlim(0, court_length)
            ax2.set_ylim(0, court_width)
            ax2.set_zlim(0, 15)

            x_court = np.array([0, court_length, court_length, 0, 0])
            y_court = np.array([0, 0, court_width, court_width, 0])
            z_court = np.zeros_like(x_court)
            ax2.plot(x_court, y_court, z_court, 'k-')

            ax2.scatter([basket_x], [basket_y], [10], color='red', s=100, label='Basket')
            ax2.scatter([feedback_data['orig_x']], [feedback_data['orig_y']], [0], color='blue', s=50, label='Original Position')

            distance = np.sqrt((feedback_data['orig_x'] - basket_x)**2 + (feedback_data['orig_y'] - basket_y)**2)
            t = np.linspace(0, 1, 20)
            x_traj = feedback_data['orig_x'] - t * (feedback_data['orig_x'] - basket_x)
            y_traj = feedback_data['orig_y'] - t * (feedback_data['orig_y'] - basket_y)
            z_traj = 4 * feedback_data['orig_max_height'] * t * (1 - t)
            ax2.plot(x_traj, y_traj, z_traj, 'b-', label='Original Trajectory')
            ax2.scatter([x_traj[10]], [y_traj[10]], [z_traj[10]], color='blue', s=30)

            # 3D Plot: Suggested trajectory
            ax3 = fig.add_subplot(224, projection='3d')
            ax3.set_title('Suggested Trajectory (3D)')
            ax3.set_xlabel('X (feet)')
            ax3.set_ylabel('Y (feet)')
            ax3.set_zlabel('Height (feet)')
            ax3.set_xlim(0, court_length)
            ax3.set_ylim(0, court_width)
            ax3.set_zlim(0, 15)

            ax3.plot(x_court, y_court, z_court, 'k-')
            ax3.scatter([basket_x], [basket_y], [10], color='red', s=100, label='Basket')
            ax3.scatter([feedback_data['sugg_x']], [feedback_data['sugg_y']], [0], color='green', s=50, label='Suggested Position')

            distance = np.sqrt((feedback_data['sugg_x'] - basket_x)**2 + (feedback_data['sugg_y'] - basket_y)**2)
            t = np.linspace(0, 1, 20)
            x_traj = feedback_data['sugg_x'] - t * (feedback_data['sugg_x'] - basket_x)
            y_traj = feedback_data['sugg_y'] - t * (feedback_data['sugg_y'] - basket_y)
            z_traj = 4 * feedback_data['sugg_max_height'] * t * (1 - t)
            ax3.plot(x_traj, y_traj, z_traj, 'g-', label='Suggested Trajectory')
            ax3.scatter([x_traj[10]], [y_traj[10]], [z_traj[10]], color='green', s=30)

            # 2D Plot: Trajectory parameters comparison
            ax4 = fig.add_subplot(223)
            ax4.set_title('Trajectory Parameters Comparison')
            param_names = ['Release Angle (°)', 'Max Height (ft)', 'Entry Angle (°)', 'Arc Ratio']
            orig_values = [np.degrees(row['release_angle']), feedback_data['orig_max_height'], feedback_data['entry_angle'], feedback_data['arc_ratio']]
            sugg_values = [feedback_data['angle'], feedback_data['sugg_max_height'], feedback_data['entry_angle'], feedback_data['arc_ratio']]

            x = np.arange(len(param_names))
            width = 0.35

            ax4.bar(x - width/2, orig_values, width, label='Original')
            ax4.bar(x + width/2, sugg_values, width, label='Suggested')

            ax4.set_ylabel('Value')
            ax4.set_xticks(x)
            ax4.set_xticklabels(param_names, rotation=45, ha='right')
            ax4.legend()

            for i, (o, s) in enumerate(zip(orig_values, sugg_values)):
                if abs(o-s) > 0.01:
                    ax4.annotate(f"{s-o:+.2f}",
                                 xy=(i + width/2, s),
                                 xytext=(0, 3),
                                 textcoords="offset points",
                                 ha='center', va='bottom',
                                 color='green' if s > o else 'red')

            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.savefig(os.path.join(output_path, f'shot_{feedback_data["shot_id"]}_feedback.png'))
            plt.close()

            print(f"Visualization for shot {shot_id} saved to {os.path.join(output_path, f'shot_{shot_id}_feedback.png')}")
            for handler in logger.handlers:
                handler.flush()
        else:
            logger.info(f"No optimal trajectory found for shot {shot_id}.")
            for handler in logger.handlers:
                handler.flush()

    print("Inference and visualization completed for all selected shots!")
    for handler in logger.handlers:
        handler.flush()
else:
    logger.info("No missed shots found in the test dataset.")
    for handler in logger.handlers:
        handler.flush()


Found 595 missed shots. Processing 10 shots.
2025-05-07 04:02:58,309 - Processing missed shot ID 0...


INFO:ShotInference:Processing missed shot ID 0...


2025-05-07 04:02:58,801 - Shot 0: Original position (x: 26.35, y: 35.55), Move to position (x: 22.72, y: 42.01), use a release angle of 36.82 degrees, aim for a maximum trajectory height of 13.41 feet, and try for an entry angle of -78.49 degrees for a 50.42% chance of making the shot.


INFO:ShotInference:Shot 0: Original position (x: 26.35, y: 35.55), Move to position (x: 22.72, y: 42.01), use a release angle of 36.82 degrees, aim for a maximum trajectory height of 13.41 feet, and try for an entry angle of -78.49 degrees for a 50.42% chance of making the shot.


Visualization for shot 0 saved to /content/drive/MyDrive/smai_project/output_graphs/shot_0_feedback.png
2025-05-07 04:02:59,964 - Processing missed shot ID 3...


...

Visualization for shot 14 saved to /content/drive/MyDrive/smai_project/output_graphs/shot_14_feedback.png
2025-05-07 04:03:10,064 - Processing missed shot ID 16...


INFO:ShotInference:Processing missed shot ID 16...


2025-05-07 04:03:10,321 - Shot 16: Original position (x: -0.10, y: 35.17), Move to position (x: 5.00, y: 33.86), use a release angle of 60.55 degrees, aim for a maximum trajectory height of 6.26 feet, and try for an entry angle of -51.07 degrees for a 52.67% chance of making the shot.


INFO:ShotInference:Shot 16: Original position (x: -0.10, y: 35.17), Move to position (x: 5.00, y: 33.86), use a release angle of 60.55 degrees, aim for a maximum trajectory height of 6.26 feet, and try for an entry angle of -51.07 degrees for a 52.67% chance of making the shot.


Visualization for shot 16 saved to /content/drive/MyDrive/smai_project/output_graphs/shot_16_feedback.png
Inference and visualization completed for all selected shots!
